In [62]:
import fitz  # PyMuPDF
import re
import spacy
import pymupdf
from sklearn.feature_extraction.text import TfidfVectorizer
import string
import pdfplumber
import pandas as pd

In [63]:
dataframe=pd.read_csv("DataSet.csv")
dataframe.head()
dataframe.paragraphs.isna().sum()

0

In [64]:
import nltk
from nltk.tokenize import word_tokenize
from nltk import ngrams
from collections import Counter
from nltk.corpus import stopwords
# nltk.download('all')

# Example sentence

def extract_all_bigrams_and_trigrams(dataframe):
# Tokenize the sentence
        list_bigrams = []
        list_trigrams = []
        stop_words = set(stopwords.words('english'))
        for i in range(len(dataframe)):
            print(i)
            tokens = word_tokenize(dataframe.paragraphs[i].lower())
            # Remove stop words from tokens
            filtered_tokens = [word for word in tokens if word.isalnum() and word not in stop_words]
            # Generate bigrams (n=2)
            bigrams = list(ngrams(filtered_tokens, 2))
            
            # print("Bigrams:", bigrams[:10])

            # Generate trigrams (n=3)
            trigrams = list(ngrams(filtered_tokens, 3))
            
            # print("Trigrams:", trigrams[:10])

            # Count the frequency of bigrams
            bigram_freq = Counter(bigrams)
            # print("Bigram Frequencies:", bigram_freq)
            trigrams_freq = Counter(trigrams)
            list_bigrams.append(bigram_freq)
            list_trigrams.append(trigrams_freq)
        return list_bigrams,list_trigrams
            # print("trigrams Frequencies:", trigrams_freq)

In [66]:
import spacy
from collections import Counter

# Load spaCy English model for POS tagging
nlp = spacy.load("en_core_web_lg")

# Step 1: Function to filter bigrams containing only nouns
def filter_ngrams_nouns(bigram_counter):
    filtered_bigrams = Counter()  # Initialize an empty Counter to store filtered bigrams
    
    for bigram, freq in bigram_counter.items():
        doc = nlp(" ".join(bigram))  # POS tag the bigram
        
        # Check if both tokens in the bigram are nouns
        if all(token.pos_ == "NOUN" for token in doc):
            filtered_bigrams[bigram] = freq  # Add only noun-based bigrams to the new Counter
    
    return filtered_bigrams


# # Step 2: Apply the filter to keep only noun-based bigrams
# filtered_bigrams = filter_ngrams_nouns(dataframe.trigram_dict[0])

# # Display the result
# print(filtered_bigrams)

In [67]:
list_bigrams,list_trigrams=extract_all_bigrams_and_trigrams(dataframe)
list_filtered_bigrams = []
list_filtered_trigrams = []
for counter in list_bigrams:
    filtered_bigrams = filter_ngrams_nouns(counter)
    list_filtered_bigrams.append(filtered_bigrams)
for counter in list_trigrams:
    filtered_trigrams = filter_ngrams_nouns(counter)
    list_filtered_trigrams.append(filtered_trigrams)

print(list_filtered_bigrams)
print(list_filtered_trigrams)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
[Counter({('risk', 'management'): 19, ('project', 'risk'): 14, ('project', 'management'): 3, ('management', 'practice'): 2, ('management', 'processes'): 2, ('plan', 'risk'): 2, ('risk', 'analysis'): 2, ('purpose', 'practice'): 1, ('management', 'practitioners'): 1, ('practitioners', 'stakeholders'): 1, ('stakeholders', 'deﬁ'): 1, ('aspects', 'project'): 1, ('practice', 'projects'): 1, ('projects', 'time'): 

In [68]:
dataframe['bigram_dict'] = list_filtered_bigrams
dataframe['trigram_dict'] = list_filtered_trigrams

In [76]:
def combine_all_bigrams(df, bigram_col):
    combined_counter = Counter()
    
    for dic_bigram in df[bigram_col]:
        combined_counter.update(dic_bigram)
    
    return combined_counter

combine_all_bigrams(dataframe, "trigram_dict")

Counter({('project', 'risk', 'management'): 248,
         ('risk', 'management', 'process'): 49,
         ('risk', 'management', 'plan'): 32,
         ('risk', 'management', 'processes'): 24,
         ('technique', 'strengths', 'weaknesses'): 20,
         ('project', 'management', 'plan'): 18,
         ('risk', 'management', 'activities'): 14,
         ('risk', 'breakdown', 'structure'): 14,
         ('risk', 'analysis', 'process'): 13,
         ('risk', 'management', 'technique'): 13,
         ('management', 'technique', 'strengths'): 13,
         ('risk', 'management', 'planning'): 12,
         ('project', 'management', 'processes'): 11,
         ('risk', 'responses', 'process'): 10,
         ('introduction', 'project', 'risk'): 7,
         ('effect', 'project', 'objectives'): 7,
         ('risk', 'action', 'owner'): 7,
         ('risk', 'management', 'project'): 6,
         ('effectiveness', 'project', 'risk'): 6,
         ('decision', 'tree', 'analysis'): 6,
         ('level', 'con

In [69]:
dataframe.head()

,sections,ids,paragraphs,bigram_dict,trigram_dict
0,1.1 Purpose of the Practice Standard for Proje...,4,The purpose of the Practice Standard for Pro...,"{('purpose', 'practice'): 1, ('project', 'risk...","{('purpose', 'practice', 'standard'): 1, ('pro..."
1,1.2 Project Risk Management Deﬁ nition,66,"The deﬁ nition of Project Risk Management, as...","{('nition', 'project'): 1, ('project', 'risk')...","{('nition', 'project', 'risk'): 1, ('project',..."
2,1.3 Role of Project Risk Management in Project...,79,Project Risk Management is not an optional ac...,"{('project', 'risk'): 11, ('risk', 'management...","{('project', 'risk', 'management'): 8, ('proje..."
3,1.4 Good Risk Management Practice,124,Project Risk Management is a valuable compone...,"{('project', 'risk'): 15, ('risk', 'management...","{('project', 'risk', 'management'): 15, ('comp..."
4,1.5 Critical Success Factors for Project Risk ...,157,Figure 1-2. Critical Success Factors for Proj...,"{('success', 'factors'): 2, ('project', 'risk'...","{('success', 'factors', 'project'): 2, ('proje..."


In [20]:

stop_words = set(stopwords.words('english'))
tokens = word_tokenize(dataframe.paragraphs[4].lower())
            # Define stop words
            

            # Remove stop words from tokens
filtered_tokens = [word for word in tokens if word.isalnum() and word not in stop_words]
            # Generate bigrams (n=2)
bigrams = list(ngrams(filtered_tokens, 2))
            
            # print("Bigrams:", bigrams[:10])

            # Generate trigrams (n=3)
trigrams = list(ngrams(filtered_tokens, 3))
            
            # print("Trigrams:", trigrams[:10])

            # Count the frequency of bigrams
bigram_freq = Counter(bigrams)
            # print("Bigram Frequencies:", bigram_freq)
trigrams_freq = Counter(trigrams)
print(bigram_freq)
print(trigrams_freq)

Counter({('risk', 'management'): 15, ('project', 'risk'): 14, ('project', 'management'): 6, ('management', 'project'): 3, ('critical', 'success'): 2, ('success', 'factors'): 2, ('factors', 'project'): 2, ('criteria', 'success'): 2, ('management', 'process'): 2, ('organizational', 'commitment'): 2, ('value', 'project'): 2, ('project', 'organization'): 2, ('management', 'processes'): 2, ('figure', 'critical'): 1, ('management', '7'): 1, ('7', 'project'): 1, ('management', 'institute'): 1, ('institute', 'practice'): 1, ('practice', 'standard'): 1, ('standard', 'project'): 1, ('management', '1'): 1, ('1', 'chapter'): 1, ('chapter', '1'): 1, ('1', 'introduction'): 1, ('introduction', 'speciﬁ'): 1, ('speciﬁ', 'c'): 1, ('c', 'criteria'): 1, ('success', 'project'): 1, ('process', 'listed'): 1, ('listed', 'chapters'): 1, ('chapters', 'dealing'): 1, ('dealing', 'processes'): 1, ('processes', 'general'): 1, ('general', 'criteria'): 1, ('success', 'include'): 1, ('include', 'recognize'): 1, ('reco

In [32]:
list_bigrams,list_trigrams=extract_all_bigrams_and_trigrams(dataframe)
dataframe['bigram_dict'] = list_bigrams
dataframe['trigram_dict'] = list_trigrams

print(len(list_bigrams))
print(len(list_trigrams))


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
175


In [33]:
dataframe.head()

,sections,ids,paragraphs,bigram_dict,trigram_dict
0,1.1 Purpose of the Practice Standard for Proje...,4,The purpose of the Practice Standard for Pro...,"{('purpose', 'practice'): 1, ('practice', 'sta...","{('purpose', 'practice', 'standard'): 1, ('pra..."
1,1.2 Project Risk Management Deﬁ nition,66,"The deﬁ nition of Project Risk Management, as...","{('deﬁ', 'nition'): 1, ('nition', 'project'): ...","{('deﬁ', 'nition', 'project'): 1, ('nition', '..."
2,1.3 Role of Project Risk Management in Project...,79,Project Risk Management is not an optional ac...,"{('project', 'risk'): 11, ('risk', 'management...","{('project', 'risk', 'management'): 8, ('risk'..."
3,1.4 Good Risk Management Practice,124,Project Risk Management is a valuable compone...,"{('project', 'risk'): 15, ('risk', 'management...","{('project', 'risk', 'management'): 15, ('risk..."
4,1.5 Critical Success Factors for Project Risk ...,157,Figure 1-2. Critical Success Factors for Proj...,"{('figure', 'critical'): 1, ('critical', 'succ...","{('figure', 'critical', 'success'): 1, ('criti..."


In [71]:
dataframe.to_csv('dataframe.csv')

In [ ]:
def combine_all_bigrams(df, bigram_col):
    combined_counter = Counter()
    
    for dic_bigram in df[bigram_col]:
        combined_counter.update(dic_bigram)
    
    return combined_counter

In [36]:
stop_words = set(stopwords.words('english'))
tokens = word_tokenize(dataframe.paragraphs[2].lower())
            # Define stop words
            

            # Remove stop words from tokens
filtered_tokens = [word for word in tokens if word.isalnum() and word not in stop_words]
            # Generate bigrams (n=2)
bigrams = list(ngrams(filtered_tokens, 2))
            
            # print("Bigrams:", bigrams[:10])

            # Generate trigrams (n=3)
trigrams = list(ngrams(filtered_tokens, 3))
fourgrams = list(ngrams(filtered_tokens, 4))
            
            # print("Trigrams:", trigrams[:10])

            # Count the frequency of bigrams
bigram_freq = Counter(bigrams)
            # print("Bigram Frequencies:", bigram_freq)
trigrams_freq = Counter(trigrams)
fourgrams_freq = Counter(fourgrams)
print(bigram_freq)
print(trigrams_freq)
print(fourgrams_freq)

Counter({('risk', 'management'): 12, ('project', 'risk'): 11, ('project', 'management'): 6, ('management', 'processes'): 5, ('project', 'plans'): 3, ('project', 'risks'): 2, ('risks', 'project'): 2, ('dates', 'critical'): 2, ('critical', 'paths'): 2, ('project', 'project'): 2, ('recognized', 'realistic'): 2, ('realistic', 'project'): 2, ('plans', 'expectations'): 2, ('expectations', 'results'): 2, ('results', 'risk'): 2, ('throughout', 'project'): 2, ('project', 'life'): 2, ('life', 'cycle'): 2, ('management', 'optional'): 1, ('optional', 'activity'): 1, ('activity', 'essential'): 1, ('essential', 'successful'): 1, ('successful', 'project'): 1, ('management', 'applied'): 1, ('applied', 'projects'): 1, ('projects', 'hence'): 1, ('hence', 'included'): 1, ('included', 'project'): 1, ('plans', 'operational'): 1, ('operational', 'documents'): 1, ('documents', 'way'): 1, ('way', 'becomes'): 1, ('becomes', 'integral'): 1, ('integral', 'part'): 1, ('part', 'every'): 1, ('every', 'aspect'): 1, 

In [51]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from collections import Counter
def filter_ngrams(ngrams, keywords):
    filtered_ngrams = [ngram for ngram in ngrams if any(kw in ngram for kw in keywords)]
    return filtered_ngrams

def preprocess_text(text):
    # Remove special characters, numbers, and extra spaces
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = text.lower().strip()
    return text

# Step 2: Extracting important n-grams based on TF-IDF
def extract_important_ngrams(paragraphs, ngram_range=(2, 3), min_df=1):
    # Preprocess paragraphs to clean text
    paragraphs = [preprocess_text(paragraph) for paragraph in paragraphs if paragraph.strip()]
    
    # Check if we still have non-empty paragraphs
    if not paragraphs:
        return []  # Return empty list if paragraphs are empty after cleaning
    
    # Use TfidfVectorizer to extract bigrams and trigrams
    vectorizer = TfidfVectorizer(ngram_range=ngram_range, stop_words='english', min_df=min_df)
    try:
        tfidf_matrix = vectorizer.fit_transform(paragraphs)
        ngrams = vectorizer.get_feature_names_out()
        return ngrams
    except ValueError:
        # Handle case where no valid n-grams are found
        return []

def get_frequent_ngrams(ngrams, threshold=5):
    ngram_freq = Counter(ngrams)
    frequent_ngrams = [ngram for ngram, count in ngram_freq.items() if count >= threshold]
    return frequent_ngrams

In [52]:
risk_keywords = ['risk', 'management', 'mitigation', 'control', 'monitor', 'impact', 'stakeholders', 'issues']

# Step 1: Extract meaningful bigrams/trigrams
dataframe['ngrams'] = dataframe.paragraphs.apply(lambda x: extract_important_ngrams([x]))

# Step 2: Filter n-grams based on risk management terms
dataframe['filtered_ngrams'] = dataframe['ngrams'].apply(lambda ngrams: filter_ngrams(ngrams, risk_keywords))


# Step 3: Get frequent n-grams as potential nodes
frequent_ngrams = get_frequent_ngrams([ngram for sublist in dataframe['filtered_ngrams'] for ngram in sublist])

# Display selected nodes (meaningful n-grams)
print(f"Selected N-grams (Nodes for Knowledge Graph): {frequent_ngrams}")
# ngrams=extract_important_ngrams(dataframe.paragraphs, ngram_range=(2, 3), min_df=0.01)
# print(ngrams[:50])

Selected N-grams (Nodes for Knowledge Graph): ['control risks', 'identify risks', 'identify risks perform', 'introduction project risk', 'management chapter', 'management chapter introduction', 'management institute', 'management institute practice', 'management processes', 'monitor control', 'monitor control risks', 'perform qualitative risk', 'perform quantitative risk', 'plan risk', 'plan risk management', 'plan risk responses', 'project management', 'project management institute', 'project risk', 'project risk management', 'qualitative risk', 'qualitative risk analysis', 'quantitative risk', 'quantitative risk analysis', 'risk analysis', 'risk management', 'risk management chapter', 'risk management processes', 'risk responses', 'risks perform', 'standard project risk', 'management planning', 'probability impact', 'risk management planning', 'level risk', 'project management processes', 'project risks', 'risk exposure', 'risk response', 'risks project', 'management plan', 'manageme

In [53]:
nlp = spacy.load('en_core_web_lg')
def filter_nouns_only(ngrams):
    filtered_ngrams = []
    
    for ngram in ngrams:
        doc = nlp(ngram)  # POS tag each n-gram using spaCy
        # Check if all tokens in the n-gram are nouns
        if all(token.pos_ == "NOUN" for token in doc):
            filtered_ngrams.append(ngram)
    
    return filtered_ngrams

In [54]:
dataframe['ngrams'] = dataframe.paragraphs.apply(lambda x: extract_important_ngrams([x]))

# Filter out n-grams that contain verbs, adverbs, or adjectives
dataframe['filtered_ngrams'] = dataframe['ngrams'].apply(filter_nouns_only)


                                            paragraphs  \
0     The purpose of the  Practice Standard for Pro...   
1     The deﬁ nition of Project Risk Management, as...   
2     Project Risk Management is not an optional ac...   
3     Project Risk Management is a valuable compone...   
4     Figure 1-2. Critical Success Factors for Proj...   
..                                                 ...   
170   The objective of risk reassessment is to ensu...   
171   Risks can and should be on the agenda at all ...   
172   The evolution of the variance values over tim...   
173   The formulae in Earned Value Analysis (CV, SV...   
174                                       no_paragraph   

                                       filtered_ngrams  
0    [analysis plan, analysis plan risk, arrangemen...  
1    [addition dimensions, advance occurrence, anal...  
2    [account instance, account instance risk, acti...  
3    [accounting law, accounting law project, agree...  
4    [actions atti

In [55]:
print(dataframe[['filtered_ngrams']])

                                       filtered_ngrams
0    [analysis plan, analysis plan risk, arrangemen...
1    [addition dimensions, advance occurrence, anal...
2    [account instance, account instance risk, acti...
3    [accounting law, accounting law project, agree...
4    [actions attitudes, approval responses, approv...
..                                                 ...
170  [control figure, control figure figure, figure...
171  [agenda items, agenda project reviews, effecti...
172  [changes respect time, cost templates, evoluti...
173  [analysis cv, basis forecasts, basis forecasts...
174                                                 []

[175 rows x 1 columns]


In [56]:
dataframe.to_csv('dataframe.csv')

Counter({('project', 'risk', 'management'): 14, ('risk', 'management', 'practice'): 2, ('risk', 'management', 'processes'): 2, ('purpose', 'practice', 'standard'): 1, ('risk', 'management', 'provide'): 1, ('project', 'management', 'practitioners'): 1, ('management', 'practitioners', 'stakeholders'): 1, ('aspects', 'project', 'risk'): 1, ('practice', 'projects', 'time'): 1, ('time', 'b', 'provide'): 1, ('edition', 'practice', 'standard'): 1, ('risk', 'programs', 'portfolios'): 1, ('edition', 'basis', 'practice'): 1, ('basis', 'practice', 'standard'): 1, ('management', 'practice', 'standard'): 1, ('edition', 'textbooks', 'handbooks'): 1, ('risk', 'management', 'resources'): 1, ('introduction', 'practice', 'standard'): 1, ('framework', 'purpose', 'principles'): 1, ('context', 'introduction', 'project'): 1, ('introduction', 'project', 'risk'): 1, ('risk', 'analysis', 'perform'): 1, ('risk', 'analysis', 'plan'): 1, ('analysis', 'plan', 'risk'): 1, ('risk', 'responses', 'monitor'): 1, ('topi